In [1]:
!python -m venv env ; 
!\source env/bin/activate

"\source" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [45]:
!pip install requests && pip install pytz
!pip install pyopenssl

In [68]:
s = requests.session()
s.keep_alive =False



In [2]:
import requests
import sqlite3

from time import sleep
from datetime import datetime
from pytz import timezone
import pandas as pd

In [29]:
#df = pd.DataFrame(columns = ['ticker','id', 'inserted', 'text', 'instruments', 'likesCount', 'nickname', 'commentsCount', 'date', 'parse_date'])
ls = []
def insert(data, ls):
    ls.append(data)
    return ls

In [5]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'accept': '*/*',
    'content-type': 'application/x-www-form-urlencoded',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
}

In [97]:
def get_data_from_api(url: str, ticker: str, cursor_number: str):
    link = url.format(ticker, cursor_number)
    session = requests.Session()
    data = session.get(link, headers=headers, stream=True)
    raw_data = data.json().get('payload').get('items')
    
    for i in tqdm(raw_data):
        my_data = {
            'ticker': ticker,
            'id': i.get('id')[:7],
            'text': i.get('text'),
            'instruments': ', '.join(get_instruments(i.get('instruments'))),
            'likesCount': i.get('likesCount'),
            'nickname': i.get('nickname'),
            'commentsCount': i.get('commentsCount'),
            'date': i.get('inserted'),
            'parse_date': datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        }
        insert(my_data, ls)
        

In [7]:
def get_cursor(url: str, ticker: str, cursor_number: str) -> str:
    link = url.format(ticker, cursor_number)
    session = requests.Session()
    data = session.get(link, headers=headers, stream=True)
    cursor_number = data.json().get('payload').get('nextCursor')
    return cursor_number

In [8]:
def get_instruments(lst: list):
    data = []
    for item in lst:
        line = f"{item.get('briefName')} !! {item.get('ticker')} !! {item.get('price')} !! {item.get('lastPrice')}"
        data.append(line)
    return data


def convert_inserted(line):
    utc_time = datetime.strptime(line, '%Y-%m-%dT%H:%M:%S.%f%z')\
        .replace(tzinfo=timezone('utc')).strftime('%Y-%m-%d %H:%M:%S')
    return utc_time

In [9]:
from tqdm import tqdm

In [126]:
link = 'https://www.tinkoff.ru/api/invest-gw/social/v1/post/instrument/{}?limit=50&appName=invest&platform=web&cursor={}'
def get_data_from_ticker(ticker: str) -> None:
    max_cursor = '99999999999'
    for _ in tqdm(range(1, 2000)):
        get_data_from_api(link, ticker, max_cursor)
        sleep(0.01)
        max_cursor = get_cursor(link, ticker, max_cursor)
        sleep(0.5)
        print('Всего данных:', len(ls))
# RUN
#get_data_from_ticker('SBER')

In [70]:
ls = []

In [95]:
securites = ['SBER', 'GAZP', 'YNDX', 'LKOH', 'TCS']
for security in securites:
    get_data_from_ticker(security)

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 12516.57it/s]

Всего данных: 7291



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 10005.97it/s]

Всего данных: 7341



100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 8336.59it/s]

Всего данных: 7391



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 10004.06it/s]

Всего данных: 7441



  0%|▏                                                                                | 4/1999 [00:06<54:36,  1.64s/it]


KeyboardInterrupt: 

In [33]:
len(ls)

106474

In [21]:
dataf = pd.DataFrame(ls)

In [28]:
dataf.inserted.min()

'2019-09-03 12:09:13'

In [25]:
dataf.to_csv('out_sber.csv')

In [49]:
dataf2 = pd.DataFrame(ls)

In [50]:
dataf2.inserted.min()

'2019-08-27 13:08:09'

In [51]:
dataf2.to_csv('out_666.csv')

In [41]:
dataf

,ticker,id,inserted,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,SBER,8ad40e8,2022-04-17 18:29:38,{$SPCE} {$VIPS} {$MOMO} {$ASTR} {$SBER} {$LKOH...,Virgin Galactic Holdings Inc Class A !! SPCE !...,0,Andreus717,10,2022-04-17T18:29:38.853309+03:00,2022-04-17 18:42:28
1,SBER,af56dc2,2022-04-17 18:21:25,📍Еврокомиссия работает над шестым санкционным ...,Сбер Банк !! SBER !! 130.88 !! 130.88,1,ShamilVaisov,0,2022-04-17T18:21:25.496598+03:00,2022-04-17 18:42:28
2,SBER,78703ee,2022-04-17 18:11:03,Забавный факт: только вчера закончился жаркий ...,"Сбер Банк !! SBER !! 130.88 !! 130.88, SBRF-6....",0,LiKimiya,6,2022-04-17T18:11:03.186447+03:00,2022-04-17 18:42:28
3,SBER,e22e9f4,2022-04-17 17:55:54,Недавно {$SGZH} написала про выплату дивидендо...,"Сегежа !! SGZH !! 8.895 !! 8.895, АФК Система ...",0,RisterSantory,1,2022-04-17T17:55:54.570007+03:00,2022-04-17 18:42:28
4,SBER,7a34c51,2022-04-17 17:52:59,Мультипликаторы и экономические сектора. \n🏦Ка...,"Банк ВТБ !! VTBR !! 0.020125 !! 0.020125, Сбер...",7,Birjevoi_Makler,0,2022-04-17T17:52:59.296846+03:00,2022-04-17 18:42:28
...,...,...,...,...,...,...,...,...,...,...
89296,SBER,77ecde2,2019-09-05 19:04:01,{$SBER} а когда отчет,Сбер Банк !! SBER !! 230.79 !! 130.88,0,ALEXX,1,2019-09-05T19:04:01.48507+03:00,2022-04-17 19:01:03
89297,SBER,2c14706,2019-09-05 17:40:27,"{$SBER}\nНу что, кто уже зафиксировал, а кто б...",Сбер Банк !! SBER !! 230.39 !! 130.88,1,MOEX4Makson,2,2019-09-05T17:40:27.454096+03:00,2022-04-17 19:01:03
89298,SBER,39303e1,2019-09-05 11:51:52,{$SBER} и {$AFKS} сегодня могут стать лидерам...,"Сбер Банк !! SBER !! 229.08 !! 130.88, АФК Сис...",5,brad,1,2019-09-05T11:51:52.250923+03:00,2022-04-17 19:01:03
89299,SBER,297da77,2019-09-04 19:28:19,"{$SBER}\n\nНу что, полетели к пятнице?",Сбер Банк !! SBER !! 227.1 !! 130.88,2,MOEX4Makson,1,2019-09-04T19:28:19.395733+03:00,2022-04-17 19:01:03


In [52]:
 len(dataf2)

127942

In [54]:
dataf2.ticker.value_counts()

SBER    49942
LKOH    25123
TCS     22644
GAZP    16636
YNDX    13597
Name: ticker, dtype: int64

In [56]:
dataf2.groupby('ticker').inserted.min()

ticker
GAZP    2022-01-19 10:59:48
LKOH    2019-09-05 14:12:22
SBER    2021-07-06 09:59:28
TCS     2019-08-27 13:08:09
YNDX    2022-01-10 21:45:48
Name: inserted, dtype: object

In [67]:
df_YNDX

,ticker,id,inserted,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,YNDX,02c1918,2022-04-17 23:36:41,Команда компании Sber Automotive Technologies ...,"Сбер Банк !! SBER !! 130.88 !! 130.88, Yandex ...",3,bizebiz,2,2022-04-17T23:36:41.14496+03:00,2022-04-18 00:45:18
1,YNDX,1a91f70,2022-04-17 23:06:12,"Как и обещал, разберем более подробно бизнес к...","VK !! VKCO !! 422.6 !! 422.6, Сбер Банк !! SBE...",9,Azot_orion,0,2022-04-17T23:06:12.493293+03:00,2022-04-18 00:45:18
2,YNDX,f593ae4,2022-04-17 20:43:15,"{$YNDX} Как думаете в теории, человек, который...",Yandex !! YNDX !! 1979.6 !! 1979.6,3,JlblC,3,2022-04-17T20:43:15.412044+03:00,2022-04-18 00:45:18
3,YNDX,1b47307,2022-04-17 19:48:26,{$YNDX} Пружина сжимается иксы неизбежны😀,Yandex !! YNDX !! 1979.6 !! 1979.6,4,MVideo,4,2022-04-17T19:48:26.710202+03:00,2022-04-18 00:45:18
4,YNDX,50791d8,2022-04-17 17:39:57,{$YNDX} До какого момента стоит покупать бумаг...,Yandex !! YNDX !! 1979.6 !! 1979.6,0,Ivan_1703,29,2022-04-17T17:39:57.224509+03:00,2022-04-18 00:45:18
...,...,...,...,...,...,...,...,...,...,...
195,YNDX,742b61d,2022-04-15 10:02:16,{$YNDX} ну а теперь вниз 😊,Yandex !! YNDX !! 1920.8 !! 1979.6,0,Vladislav5959,1,2022-04-15T10:02:16.700108+03:00,2022-04-18 00:45:19
196,YNDX,855eabe,2022-04-15 09:57:24,{$YNDX} при открытие вверх по стакану,Yandex !! YNDX !! 1945.2 !! 1979.6,0,Vladislav5959,3,2022-04-15T09:57:24.745082+03:00,2022-04-18 00:45:19
197,YNDX,92e3686,2022-04-15 09:56:11,{$YNDX} читал что у Яши долгов много и чтобы з...,Yandex !! YNDX !! 1945.2 !! 1979.6,1,Xisma93,1,2022-04-15T09:56:11.510421+03:00,2022-04-18 00:45:19
198,YNDX,247363d,2022-04-15 09:55:52,"{$YNDX} если ниже 1900 уйдёт, негра девушке на...",Yandex !! YNDX !! 1945.2 !! 1979.6,0,Ebelman,5,2022-04-15T09:55:52.813569+03:00,2022-04-18 00:45:19


In [101]:
ls = []
get_data_from_ticker('GAZP')


100%|███████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 24557.40it/s]

Всего данных: 50



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 49955.98it/s]

Всего данных: 99



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 25007.77it/s]

...

Всего данных: 71491



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 49884.68it/s]

Всего данных: 71541



 72%|██████████████████████████████████████████████████████▍                     | 1431/1999 [1:41:01<40:06,  4.24s/it]


SSLError: HTTPSConnectionPool(host='www.tinkoff.ru', port=443): Max retries exceeded with url: /api/invest-gw/social/v1/post/instrument/GAZP?limit=50&appName=invest&platform=web&cursor=272269 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')")))

In [104]:
df_GAZP = pd.DataFrame(ls)
df_GAZP
df_GAZP.to_csv('out_GAZP.csv')

In [78]:
df_GAZP

,ticker,id,inserted,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,GAZP,59a81ca,2022-04-18 00:39:39,{$GAZP}\n#Вучич #исключительные_условия\n#Евро...,Газпром !! GAZP !! 224.0 !! 224.0,0,basebel,0,2022-04-18T00:39:39.153761+03:00,2022-04-18 00:56:44
1,GAZP,6c1761c,2022-04-17 23:07:59,Одна акция - {$GAZP}\nОдин урок - один раз слу...,Газпром !! GAZP !! 224.0 !! 224.0,2,yeezzzzyy,0,2022-04-17T23:07:59.703769+03:00,2022-04-18 00:56:44
2,GAZP,427a732,2022-04-17 22:32:22,#daytrading #trading #news \n\nВажные новости...,"Газпром !! GAZP !! 224.0 !! 224.0, НОВАТЭК !! ...",1,Sege1992,0,2022-04-17T22:32:22.422541+03:00,2022-04-18 00:56:44
3,GAZP,74f5271,2022-04-17 22:01:10,По оценкам bloomberg Россия в этом году зарабо...,"Газпром !! GAZP !! 224.0 !! 224.0, ЛУКОЙЛ !! L...",2,POPOWADIM,1,2022-04-17T22:01:10.557649+03:00,2022-04-18 00:56:44
4,GAZP,e791b25,2022-04-17 21:31:20,{$GAZP} {$SBER} #мотивация #ошибки #инвестици...,"Газпром !! GAZP !! 224.0 !! 224.0, Сбер Банк !...",15,Ruslan_Invest92,4,2022-04-17T21:31:20.886159+03:00,2022-04-18 00:56:44
...,...,...,...,...,...,...,...,...,...,...
16636,GAZP,65ae34b,2022-01-19 11:02:28,{$GAZP} Цена войны: как геополитика губит экон...,Газпром !! GAZP !! 308.35 !! 224.0,3,NoVic1959,20,2022-01-19T11:02:28.848592+03:00,2022-04-18 01:05:07
16637,GAZP,4b34461,2022-01-19 11:01:52,{$GAZP} бегите глупцы,Газпром !! GAZP !! 308.35 !! 224.0,1,Dedkukan,1,2022-01-19T11:01:52.656806+03:00,2022-04-18 01:05:07
16638,GAZP,2f552a6,2022-01-19 11:01:08,{$GAZP} вниз,Газпром !! GAZP !! 309.08 !! 224.0,3,iibragin,0,2022-01-19T11:01:08.155037+03:00,2022-04-18 01:05:07
16639,GAZP,4706918,2022-01-19 11:00:55,#новости #события #Россия \n{$YNDX} S&P присво...,"Yandex !! YNDX !! 3808.8 !! 1979.6, VK !! VKCO...",19,Money_Way,0,2022-01-19T11:00:55.30868+03:00,2022-04-18 01:05:07


In [73]:
ls = []
get_data_from_ticker('YNDX')
df_YNDX = pd.DataFrame(ls)
df_YNDX.to_csv('out_YNDX.csv')

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 12507.62it/s]

Всего данных: 50



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 12505.38it/s]

Всего данных: 100



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 10005.50it/s]

...

Всего данных: 41097



100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 8338.25it/s]

Всего данных: 41147



 41%|████████████████████████████████▌                                              | 823/1999 [21:06<30:09,  1.54s/it]


SSLError: HTTPSConnectionPool(host='www.tinkoff.ru', port=443): Max retries exceeded with url: /api/invest-gw/social/v1/post/instrument/YNDX?limit=100&appName=invest&platform=web&cursor=1150359 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')")))

In [75]:
df_YNDX = pd.DataFrame(ls)


In [76]:
df_YNDX

,ticker,id,inserted,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,YNDX,02c1918,2022-04-17 23:36:41,Команда компании Sber Automotive Technologies ...,"Сбер Банк !! SBER !! 130.88 !! 130.88, Yandex ...",4,bizebiz,2,2022-04-17T23:36:41.14496+03:00,2022-04-18 01:05:09
1,YNDX,1a91f70,2022-04-17 23:06:12,"Как и обещал, разберем более подробно бизнес к...","VK !! VKCO !! 422.6 !! 422.6, Сбер Банк !! SBE...",9,Azot_orion,0,2022-04-17T23:06:12.493293+03:00,2022-04-18 01:05:09
2,YNDX,f593ae4,2022-04-17 20:43:15,"{$YNDX} Как думаете в теории, человек, который...",Yandex !! YNDX !! 1979.6 !! 1979.6,3,JlblC,3,2022-04-17T20:43:15.412044+03:00,2022-04-18 01:05:09
3,YNDX,1b47307,2022-04-17 19:48:26,{$YNDX} Пружина сжимается иксы неизбежны😀,Yandex !! YNDX !! 1979.6 !! 1979.6,4,MVideo,4,2022-04-17T19:48:26.710202+03:00,2022-04-18 01:05:09
4,YNDX,50791d8,2022-04-17 17:39:57,{$YNDX} До какого момента стоит покупать бумаг...,Yandex !! YNDX !! 1979.6 !! 1979.6,0,Ivan_1703,29,2022-04-17T17:39:57.224509+03:00,2022-04-18 01:05:09
...,...,...,...,...,...,...,...,...,...,...
41192,YNDX,0dc3f0d,2020-09-23 16:35:33,{$YNDX} теперь вниз⬇️😆,Yandex !! YNDX !! 5223.2 !! 1979.6,0,Ruz01,1,2020-09-23T16:35:33.535768+03:00,2022-04-18 01:26:06
41193,YNDX,c30006e,2020-09-23 16:35:17,{$YNDX},Yandex !! YNDX !! 5223.2 !! 1979.6,2,Le__Chiffre,0,2020-09-23T16:35:17.631773+03:00,2022-04-18 01:26:06
41194,YNDX,861e8e5,2020-09-23 16:35:13,{$TCS} {$YNDX} Яндекс» подорожал на $3 млрд н...,TCS Group (Tinkoff Bank holder) !! TCS !! 27.7...,2,Babkirulyat,0,2020-09-23T16:35:13.017932+03:00,2022-04-18 01:26:06
41195,YNDX,d631ccd,2020-09-23 16:35:12,{$YNDX} сливайте пока не поздно 🤞🏻😂🤦🏻,Yandex !! YNDX !! 5223.2 !! 1979.6,3,Profit_Sly_Fox,3,2020-09-23T16:35:12.623395+03:00,2022-04-18 01:26:06


In [77]:
df_YNDX.to_csv('out_YNDX.csv')

In [131]:
ls = []
get_data_from_ticker('GMKN')
df_GMKN = pd.DataFrame(ls)
df_GMKN.to_csv('out_GMKN.csv')

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 50412.31it/s]

Всего данных: 49



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 25016.72it/s]

Всего данных: 99



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 25013.74it/s]

...


 28%|██████████████████████                                                         | 558/1999 [06:57<17:24,  1.38it/s]
0it [00:00, ?it/s]

Всего данных: 27878



 28%|██████████████████████                                                         | 559/1999 [06:58<16:50,  1.43it/s]
0it [00:00, ?it/s]
 28%|██████████████████████                                                         | 559/1999 [06:58<17:57,  1.34it/s]

Всего данных: 27878


TypeError: 'NoneType' object is not iterable

In [136]:
df_GMKN = pd.DataFrame(ls)
df_GMKN

,ticker,id,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,GMKN,1e792b9,Прогноз Fitch относительно цен на медь\n\nПере...,"Freeport-McMoRan !! FCX !! 50.16 !! 50.05, Val...",2,KawaBaggio,0,2022-04-18T16:50:47.502918+03:00,2022-04-18 17:11:49
1,GMKN,dcaffa1,Разведка по открытым источникам. Генерал-лейте...,"Газпром !! GAZP !! 218.5 !! 218.58, Норильский...",2,19K,1,2022-04-18T15:33:03.25823+03:00,2022-04-18 17:11:49
2,GMKN,41b5dae,"Несмотря на санкции, бизнес хочет платить по д...","Северсталь !! CHMF !! 975.2 !! 974.6, Норильск...",2,FromCrimea,4,2022-04-18T15:15:26.418734+03:00,2022-04-18 17:11:49
3,GMKN,149902d,{$GMKN} куда же эта ракета летит,Норильский никель !! GMKN !! 19850.0 !! 19954.0,0,takahuli,0,2022-04-18T15:01:55.81619+03:00,2022-04-18 17:11:49
4,GMKN,9d3abc5,{$GMKN} Суд решил вернуть государству Норильс...,Норильский никель !! GMKN !! 19980.0 !! 19954.0,2,investment_177,0,2022-04-18T14:30:25.600478+03:00,2022-04-18 17:11:49
...,...,...,...,...,...,...,...,...,...
27873,GMKN,4775ee2,{$GMKN} ждём рост к дивам,Норильский никель !! GMKN !! 16048 !! 19822.0,1,bank,0,2019-09-12T10:09:36.781344+03:00,2022-04-18 17:18:46
27874,GMKN,9807495,{$GMKN} Коррекция или всё-таки сменили тренд? ...,Норильский никель !! GMKN !! 16038 !! 19822.0,3,Kristine,3,2019-09-09T13:38:22.119255+03:00,2022-04-18 17:18:46
27875,GMKN,534709c,{$GMKN} ты кудаааа????😳😳😳,Норильский никель !! GMKN !! 16300 !! 19822.0,2,mamka74,1,2019-09-05T14:04:06.967501+03:00,2022-04-18 17:18:46
27876,GMKN,739317e,{$GMKN} шортанем на пару процентов за сегодня...,Норильский никель !! GMKN !! 16432 !! 19822.0,3,Toha74,6,2019-09-04T13:41:05.264504+03:00,2022-04-18 17:18:46


In [127]:
#ls = []
get_data_from_ticker('VTBR')

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 50027.48it/s]

Всего данных: 16081



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 24986.92it/s]

Всего данных: 16131



100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 50003.62it/s]

...


 22%|█████████████████                                                              | 431/1999 [05:29<19:16,  1.36it/s]
0it [00:00, ?it/s]

Всего данных: 37574



 22%|█████████████████                                                              | 432/1999 [05:30<18:40,  1.40it/s]
0it [00:00, ?it/s]
 22%|█████████████████                                                              | 432/1999 [05:30<19:59,  1.31it/s]

Всего данных: 37574


TypeError: 'NoneType' object is not iterable

In [128]:
df_VTBR = pd.DataFrame(ls)
df_VTBR.drop(columns = 'parse_date').duplicated().sum()

646

In [130]:
df_VTBR.to_csv('out_VTBR.csv')

In [125]:
df_GAZP.drop(columns = 'parse_date').duplicated().sum()

0

In [137]:
df = pd.read_csv('out_666.csv')

In [139]:
df = df.drop(columns = 'inserted')

In [142]:
df_SBER = pd.read_csv('out_SBER.csv')

In [144]:
df_SBER = df_SBER.drop(columns = 'inserted')

In [145]:
df_VTBR

,ticker,id,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,VTBR,b2d0baa,{$TCSG}\nПутин внес изменения в ряд законодате...,"TCS Group !! TCSG !! 2795.0 !! 2786.5, TCS Gro...",1,Adick,1,2022-04-18T16:52:11.466037+03:00,2022-04-18 16:56:57
1,VTBR,0e67638,{$VTBR} {$SBER} как вы думаете после спец опер...,"Банк ВТБ !! VTBR !! 0.0195 !! 0.019465, Сбер Б...",0,EpilepticKid,7,2022-04-18T16:49:42.719521+03:00,2022-04-18 16:56:57
2,VTBR,43137b7,Путин внес изменения в ряд законодательных акт...,TCS Group (Tinkoff Bank holder) !! TCS !! 36.2...,10,Veles_Amulet,0,2022-04-18T16:36:35.043517+03:00,2022-04-18 16:56:57
3,VTBR,d9ce559,{$VTBR} фу... \nМинус 1.000.000 на этом г...е\...,Банк ВТБ !! VTBR !! 0.019535 !! 0.019465,1,lariolga,2,2022-04-18T16:36:00.834875+03:00,2022-04-18 16:56:57
4,VTBR,57f2347,📝Сейчас рынок находится ни в лучшей стадии: об...,"Доллар США !! USDRUB !! 78.89 !! 78.9, Ozon Ho...",6,Invest_trade1,3,2022-04-18T16:36:00.159969+03:00,2022-04-18 16:56:57
...,...,...,...,...,...,...,...,...,...
37569,VTBR,ae44a56,{$VTBR} кажется коррекция успешного старта под...,Банк ВТБ !! VTBR !! 0.042045 !! 0.01937,1,Kachmen,0,2019-09-05T16:25:48.991157+03:00,2022-04-18 17:09:51
37570,VTBR,4d88425,"{$VTBR} не закрыты гэпы, наблюдаем",Банк ВТБ !! VTBR !! 0.042435 !! 0.01937,2,fos4,0,2019-09-05T14:02:27.43762+03:00,2022-04-18 17:09:51
37571,VTBR,21c4d7e,{$VTBR} ВТБ взлетел на фоне покупки части рос...,Банк ВТБ !! VTBR !! 0.04291 !! 0.01937,2,brad,3,2019-09-05T00:57:30.994063+03:00,2022-04-18 17:09:51
37572,VTBR,d163893,{$VTBR} думаю к 2022 году цена акций удвоится,Банк ВТБ !! VTBR !! 0.04291 !! 0.01937,0,RiskyGuy,1,2019-09-04T18:54:19.201482+03:00,2022-04-18 17:09:51


In [146]:
df_YNDX

,ticker,id,inserted,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,YNDX,02c1918,2022-04-17 23:36:41,Команда компании Sber Automotive Technologies ...,"Сбер Банк !! SBER !! 130.88 !! 130.88, Yandex ...",4,bizebiz,2,2022-04-17T23:36:41.14496+03:00,2022-04-18 01:05:09
1,YNDX,1a91f70,2022-04-17 23:06:12,"Как и обещал, разберем более подробно бизнес к...","VK !! VKCO !! 422.6 !! 422.6, Сбер Банк !! SBE...",9,Azot_orion,0,2022-04-17T23:06:12.493293+03:00,2022-04-18 01:05:09
2,YNDX,f593ae4,2022-04-17 20:43:15,"{$YNDX} Как думаете в теории, человек, который...",Yandex !! YNDX !! 1979.6 !! 1979.6,3,JlblC,3,2022-04-17T20:43:15.412044+03:00,2022-04-18 01:05:09
3,YNDX,1b47307,2022-04-17 19:48:26,{$YNDX} Пружина сжимается иксы неизбежны😀,Yandex !! YNDX !! 1979.6 !! 1979.6,4,MVideo,4,2022-04-17T19:48:26.710202+03:00,2022-04-18 01:05:09
4,YNDX,50791d8,2022-04-17 17:39:57,{$YNDX} До какого момента стоит покупать бумаг...,Yandex !! YNDX !! 1979.6 !! 1979.6,0,Ivan_1703,29,2022-04-17T17:39:57.224509+03:00,2022-04-18 01:05:09
...,...,...,...,...,...,...,...,...,...,...
41192,YNDX,0dc3f0d,2020-09-23 16:35:33,{$YNDX} теперь вниз⬇️😆,Yandex !! YNDX !! 5223.2 !! 1979.6,0,Ruz01,1,2020-09-23T16:35:33.535768+03:00,2022-04-18 01:26:06
41193,YNDX,c30006e,2020-09-23 16:35:17,{$YNDX},Yandex !! YNDX !! 5223.2 !! 1979.6,2,Le__Chiffre,0,2020-09-23T16:35:17.631773+03:00,2022-04-18 01:26:06
41194,YNDX,861e8e5,2020-09-23 16:35:13,{$TCS} {$YNDX} Яндекс» подорожал на $3 млрд н...,TCS Group (Tinkoff Bank holder) !! TCS !! 27.7...,2,Babkirulyat,0,2020-09-23T16:35:13.017932+03:00,2022-04-18 01:26:06
41195,YNDX,d631ccd,2020-09-23 16:35:12,{$YNDX} сливайте пока не поздно 🤞🏻😂🤦🏻,Yandex !! YNDX !! 5223.2 !! 1979.6,3,Profit_Sly_Fox,3,2020-09-23T16:35:12.623395+03:00,2022-04-18 01:26:06


In [147]:
df_GMKN

,ticker,id,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,GMKN,1e792b9,Прогноз Fitch относительно цен на медь\n\nПере...,"Freeport-McMoRan !! FCX !! 50.16 !! 50.05, Val...",2,KawaBaggio,0,2022-04-18T16:50:47.502918+03:00,2022-04-18 17:11:49
1,GMKN,dcaffa1,Разведка по открытым источникам. Генерал-лейте...,"Газпром !! GAZP !! 218.5 !! 218.58, Норильский...",2,19K,1,2022-04-18T15:33:03.25823+03:00,2022-04-18 17:11:49
2,GMKN,41b5dae,"Несмотря на санкции, бизнес хочет платить по д...","Северсталь !! CHMF !! 975.2 !! 974.6, Норильск...",2,FromCrimea,4,2022-04-18T15:15:26.418734+03:00,2022-04-18 17:11:49
3,GMKN,149902d,{$GMKN} куда же эта ракета летит,Норильский никель !! GMKN !! 19850.0 !! 19954.0,0,takahuli,0,2022-04-18T15:01:55.81619+03:00,2022-04-18 17:11:49
4,GMKN,9d3abc5,{$GMKN} Суд решил вернуть государству Норильс...,Норильский никель !! GMKN !! 19980.0 !! 19954.0,2,investment_177,0,2022-04-18T14:30:25.600478+03:00,2022-04-18 17:11:49
...,...,...,...,...,...,...,...,...,...
27873,GMKN,4775ee2,{$GMKN} ждём рост к дивам,Норильский никель !! GMKN !! 16048 !! 19822.0,1,bank,0,2019-09-12T10:09:36.781344+03:00,2022-04-18 17:18:46
27874,GMKN,9807495,{$GMKN} Коррекция или всё-таки сменили тренд? ...,Норильский никель !! GMKN !! 16038 !! 19822.0,3,Kristine,3,2019-09-09T13:38:22.119255+03:00,2022-04-18 17:18:46
27875,GMKN,534709c,{$GMKN} ты кудаааа????😳😳😳,Норильский никель !! GMKN !! 16300 !! 19822.0,2,mamka74,1,2019-09-05T14:04:06.967501+03:00,2022-04-18 17:18:46
27876,GMKN,739317e,{$GMKN} шортанем на пару процентов за сегодня...,Норильский никель !! GMKN !! 16432 !! 19822.0,3,Toha74,6,2019-09-04T13:41:05.264504+03:00,2022-04-18 17:18:46


In [148]:
df_GAZP

,ticker,id,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,GAZP,59a81ca,{$GAZP}\n#Вучич #исключительные_условия\n#Евро...,Газпром !! GAZP !! 224.0 !! 224.0,0,basebel,0,2022-04-18T00:39:39.153761+03:00,2022-04-18 04:01:14
1,GAZP,6c1761c,Одна акция - {$GAZP}\nОдин урок - один раз слу...,Газпром !! GAZP !! 224.0 !! 224.0,2,yeezzzzyy,1,2022-04-17T23:07:59.703769+03:00,2022-04-18 04:01:14
2,GAZP,427a732,#daytrading #trading #news \n\nВажные новости...,"Газпром !! GAZP !! 224.0 !! 224.0, НОВАТЭК !! ...",1,Sege1992,0,2022-04-17T22:32:22.422541+03:00,2022-04-18 04:01:14
3,GAZP,74f5271,По оценкам bloomberg Россия в этом году зарабо...,"Газпром !! GAZP !! 224.0 !! 224.0, ЛУКОЙЛ !! L...",2,POPOWADIM,1,2022-04-17T22:01:10.557649+03:00,2022-04-18 04:01:14
4,GAZP,e791b25,{$GAZP} {$SBER} #мотивация #ошибки #инвестици...,"Газпром !! GAZP !! 224.0 !! 224.0, Сбер Банк !...",16,Ruslan_Invest92,5,2022-04-17T21:31:20.886159+03:00,2022-04-18 04:01:14
...,...,...,...,...,...,...,...,...,...
71586,GAZP,9eb1999,"Как стать проф трейдером, учитесь у меня:\nБер...",Starbucks Corporation !! SBUX !! 61.62 !! 79.5...,43,jey499,29,2020-03-19T22:53:16.918486+03:00,2022-04-18 05:42:04
71587,GAZP,4a7c546,"{$GAZP} все супер, экономика восстанавливается...",Газпром !! GAZP !! 174.56 !! 224.0,2,Wasnt,11,2020-03-19T22:15:01.588854+03:00,2022-04-18 05:42:04
71588,GAZP,aaf473a,​​Эти недели войдут в историю...\n\nСтолько но...,"AT&T !! T !! 31.36 !! 19.53, Tesla Motors !! T...",105,cherkasov_va,21,2020-03-19T20:37:45.080491+03:00,2022-04-18 05:42:04
71589,GAZP,5658a1f,{$GAZP} хочу завтра купить Газпром. Стоит?,Газпром !! GAZP !! 174.56 !! 224.0,1,Dmitry_web,4,2020-03-19T20:31:32.111359+03:00,2022-04-18 05:42:04


In [149]:
df = df.append(df_VTBR)

In [150]:
df = df[df.ticker.isin(['LKOH','TCS','VTBR'])]

In [152]:
df = df.drop(columns=['Unnamed: 0'])

In [153]:
df

,ticker,id,text,instruments,likesCount,nickname,commentsCount,date,parse_date
80175,LKOH,fb9bdac,В ожидании понедельника испущено много нервов ...,"Московская Биржа !! MOEX !! 93.65 !! 93.65, ЛУ...",8,NalcoProxy,13,2022-04-17T19:01:09.640148+03:00,2022-04-17 20:53:17
80176,LKOH,8ad40e8,{$SPCE} {$VIPS} {$MOMO} {$ASTR} {$SBER} {$LKOH...,Virgin Galactic Holdings Inc Class A !! SPCE !...,1,Andreus717,43,2022-04-17T18:29:38.853309+03:00,2022-04-17 20:53:17
80177,LKOH,1746a1b,⚡️Британцы все меньше поддерживают антироссийс...,"НОВАТЭК !! NVTK !! 1056.0 !! 1056.0, Газпром н...",1,Lirian,1,2022-04-17T17:37:37.519304+03:00,2022-04-17 20:53:17
80178,LKOH,b43e796,#УрокиВПульсе \n\nБыло это на самой заре моего...,ЛУКОЙЛ !! LKOH !! 4893.0 !! 4893.0,6,Genitive,0,2022-04-17T14:17:28.139543+03:00,2022-04-17 20:53:17
80179,LKOH,0fb444e,Решил посчитать доходность акций МТС {$MTSS} э...,"МТС !! MTSS !! 207.35 !! 207.35, Доллар США !!...",7,bizebiz,8,2022-04-17T09:39:05.899903+03:00,2022-04-17 20:53:17
...,...,...,...,...,...,...,...,...,...
37569,VTBR,ae44a56,{$VTBR} кажется коррекция успешного старта под...,Банк ВТБ !! VTBR !! 0.042045 !! 0.01937,1,Kachmen,0,2019-09-05T16:25:48.991157+03:00,2022-04-18 17:09:51
37570,VTBR,4d88425,"{$VTBR} не закрыты гэпы, наблюдаем",Банк ВТБ !! VTBR !! 0.042435 !! 0.01937,2,fos4,0,2019-09-05T14:02:27.43762+03:00,2022-04-18 17:09:51
37571,VTBR,21c4d7e,{$VTBR} ВТБ взлетел на фоне покупки части рос...,Банк ВТБ !! VTBR !! 0.04291 !! 0.01937,2,brad,3,2019-09-05T00:57:30.994063+03:00,2022-04-18 17:09:51
37572,VTBR,d163893,{$VTBR} думаю к 2022 году цена акций удвоится,Банк ВТБ !! VTBR !! 0.04291 !! 0.01937,0,RiskyGuy,1,2019-09-04T18:54:19.201482+03:00,2022-04-18 17:09:51


In [159]:
df.groupby('ticker').date.min()

ticker
GAZP    2020-03-19T19:53:23.358428+03:00
GMKN    2019-09-04T11:52:10.200386+03:00
LKOH    2019-09-05T14:12:22.389524+03:00
SBER    2019-09-03T12:09:13.423684+03:00
TCS     2019-08-27T13:08:09.900498+03:00
VTBR    2019-09-04T15:14:42.184673+03:00
YNDX    2020-09-23T16:35:05.650301+03:00
Name: date, dtype: object

In [160]:
df.ticker.value_counts()

SBER    89301
GAZP    71591
YNDX    41197
VTBR    37574
GMKN    27878
LKOH    25123
TCS     22644
Name: ticker, dtype: int64

In [157]:
df = df.append(df_GMKN)
df = df.append(df_GAZP)
df = df.append(df_SBER)
df = df.append(df_YNDX)

In [162]:
df.to_csv('final_out_diploma.csv', index = False)

In [163]:
df = df.reset_index(drop=True)

In [4]:
df = pd.read_csv('final_out_diploma.csv')

C:\Users\03\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df = df.drop(columns = ['Unnamed: 0', 'inserted'])

In [10]:
df.to_csv('final_out_diploma.csv', index = False)

In [11]:
df

,ticker,id,text,instruments,likesCount,nickname,commentsCount,date,parse_date
0,LKOH,fb9bdac,В ожидании понедельника испущено много нервов ...,"Московская Биржа !! MOEX !! 93.65 !! 93.65, ЛУ...",8,NalcoProxy,13,2022-04-17T19:01:09.640148+03:00,2022-04-17 20:53:17
1,LKOH,8ad40e8,{$SPCE} {$VIPS} {$MOMO} {$ASTR} {$SBER} {$LKOH...,Virgin Galactic Holdings Inc Class A !! SPCE !...,1,Andreus717,43,2022-04-17T18:29:38.853309+03:00,2022-04-17 20:53:17
2,LKOH,1746a1b,⚡️Британцы все меньше поддерживают антироссийс...,"НОВАТЭК !! NVTK !! 1056.0 !! 1056.0, Газпром н...",1,Lirian,1,2022-04-17T17:37:37.519304+03:00,2022-04-17 20:53:17
3,LKOH,b43e796,#УрокиВПульсе \n\nБыло это на самой заре моего...,ЛУКОЙЛ !! LKOH !! 4893.0 !! 4893.0,6,Genitive,0,2022-04-17T14:17:28.139543+03:00,2022-04-17 20:53:17
4,LKOH,0fb444e,Решил посчитать доходность акций МТС {$MTSS} э...,"МТС !! MTSS !! 207.35 !! 207.35, Доллар США !!...",7,bizebiz,8,2022-04-17T09:39:05.899903+03:00,2022-04-17 20:53:17
...,...,...,...,...,...,...,...,...,...
315303,YNDX,0dc3f0d,{$YNDX} теперь вниз⬇️😆,Yandex !! YNDX !! 5223.2 !! 1979.6,0,Ruz01,1,2020-09-23T16:35:33.535768+03:00,2022-04-18 01:26:06
315304,YNDX,c30006e,{$YNDX},Yandex !! YNDX !! 5223.2 !! 1979.6,2,Le__Chiffre,0,2020-09-23T16:35:17.631773+03:00,2022-04-18 01:26:06
315305,YNDX,861e8e5,{$TCS} {$YNDX} Яндекс» подорожал на $3 млрд н...,TCS Group (Tinkoff Bank holder) !! TCS !! 27.7...,2,Babkirulyat,0,2020-09-23T16:35:13.017932+03:00,2022-04-18 01:26:06
315306,YNDX,d631ccd,{$YNDX} сливайте пока не поздно 🤞🏻😂🤦🏻,Yandex !! YNDX !! 5223.2 !! 1979.6,3,Profit_Sly_Fox,3,2020-09-23T16:35:12.623395+03:00,2022-04-18 01:26:06


In [14]:
sample_for_razmetka = df[['ticker','text']].sample(n=10000, random_state=1488)

In [15]:
sample_for_razmetka.to_csv('sample.csv')

In [16]:
sample_for_razmetka

,ticker,text
212461,SBER,"{$SBER} Едет катится, на 261 вагон\nЖулья в нё..."
225668,SBER,{$SBER} Иностранцы специально сливают?) Может...
216978,SBER,{$TCSG} {$SBER} {$GAZP} \nСмешно наблюдать как...
139619,GAZP,{$GAZP} интересно производство вакцин это энер...
313133,YNDX,"{$YNDX} жесть конечно,а еще два дня назад было..."
...,...,...
219584,SBER,{$SBER} слишком быстро и рано по времени отско...
307428,YNDX,{$YNDX} серьезные плиты по 4.5 млн в стакане 🙄
158632,GAZP,Нефтяные цены в ходе торгов продолжают повышат...
237051,SBER,​🔥Итоги дня: IMOEX -1.3%. Дивидендная отсечка\...


In [17]:
sample_for_razmetka.to_excel('sample.xlsx', encoding = 'UTF-8')